In [3]:
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn.preprocessing 
import sklearn.metrics
import matplotlib.pyplot as plt 
from scipy.signal import butter, lfilter
import csv
from datetime import datetime

# This part is for data preprocessing
### input: 
The data stream (sensor data)
### output: 
The np.array with the format of deltaT, the previous data, and the label is the next max

In [7]:
physical_devices = tf.config.experimental.list_physical_devices('GPU') #physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  0


IndexError: list index out of range

# Convention for datasets
The base time step is in milliseconds

The order of acceleration axes is by default (x,y,z) and the data is by default selected according to the x axis. To make modifications to this configuration, command-line flags are available

In [1]:
def import_data(path_to_dataset,seperator=',',sampling_freq=1000,order=['ax','ay','az','time'],abstime=0,stamp=0): 
    #imports data, seperated by a comma (default), with a 1 kHz frequency
    #the order of the data columns by default ax, ay, az, and are the timestamps provided              (abstime=1) or are timestamps in delta times and a list of data to keep
    data_init = pd.read_csv(filepath_or_buffer=path_to_dataset,sep=seperator, header=0,names=order)
    data_init.header('dt')
    if data_init.header('dt') != None:
        #Adding A cummulative sum comlun(for absolute time), this will serve as identification and         will help during plotting
        data_init["time"]=data_init["dt"].cumsum()
    #----------------------------------------
    # 
    # if stamp:
    #To generate a time stamp 
    #    data_init["timestamp"]= data_init["time"].apply(lambda x: datetime(microseconds = x*))
    #
    #----------------------------------------

    #data_init[][len(data_init)-1]
    data_init.drop(headers=["az","ay"])

    data_out=pd.headers(data_init.headers())
    #Homogenizing the sampling rate (we settle for the case where delta t is 1)
    num_cols = len(data_init.columns())
    j=0
    for index in len(data_init):
        if data_init["dt"].loc(index) == 1:
            data_out.loc[j] =data_init[index]
            j+=1
        else:
            for i in range(0,data_init["dt"].loc()):
                data_out.loc[j] = [np.NaN for p in range(0,num_cols)]
                j+=1
            data_out.loc[j] =data_init[index]
            j+=1
    data_init.drop(headers=["dt"])
    return data_out

IndentationError: expected an indented block (<ipython-input-1-5e10f8f73bab>, line 11)

In [ ]:
def probe(df):
    print("-------Information about the dataframe-------")
    df.info()
    df.describe()
    print("-------Head and Tail of the dataframe-------")
    df.Head()
    df.Tail()
    if df.value_counts():
        print("-------Count of missing data in the dataframe-------")
        df.value_counts()


In [ ]:
def scope(xaxis,xlabel="",yaxis,ylabel="",step=1, join=0):
    #takes in an xaxis and a yaxis(could contain headers)
    if len(yaxis.columns())-1:
        +

In [ ]:
def filter(method='linear',df,fs=1000,cutoff,order=2):
    #takes in a dataframe, a sampling frequency (1 kHz by default), a cutoff frquency, and an order and applies a lowpass butterworth filter to the acceleration data
    df["ax"]=df["ax"].interpolate(method=method, limit_direction='forward')
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    data=df["ax"].to_numpy()
    y = lfilter(b, a, data)
    data_out =pd.dataframe(y)
    return data_out

In [ ]:
def extremes(df, fs):
    #determining the extrema 
    df['min'] = df['ax'][(df['ax'].shift(1) > df['ax']) & (df['ax'].shift(-1) > df['ax'])]
    df['max'] = df['ax'][(df['ax'].shift(1) < df['ax']) & (df['ax'].shift(-1) < df['ax'])]
    #computing the difference between consecutive extrema as well as the time difference between each extreme  (following)
    min=df['ax'].loc[0]
    max=df['ax'].loc[0]
    delta_t=0
    i_last_min=0
    i_last_max=0
    for index in len(df)
        if df['min'].loc[index]:
            min= df['min'].loc[index]
            df['delta_ext'].loc[index]=max-min
            df['next_ext'].loc[i_last_min] = delta_t
            delta_t=0
            i_last_min=index
        if df['max'].loc[index]:
            max= df['max'].loc[index]
            df['delta_ext'].loc[index]=max-min 
            df['next_ext'].loc[i_last_max] = delta_t
            delta_t=0
            i_last_min=index
        delta_t+= 1/fs
        #and then you fill the empty slots with a linear interpolation of the time 
    return df


In [ ]:
def interpolate(df):
    #Introduce the colum
    df=df.interpolate(method='spline',order=5, limit_direction='forward')
    return df

In [ ]:
def train_val_test_(df,headers,outputs, win, train_ratio):
#Manipulates the entry data into tensors of window size and then the model output should be min max and delta_next to provide history for each: df (datafame), HEADER (string of the column to reshape), win (the past values) 
#make a tensor (window(),2,len(sequence))
    for i in range(0, win):
        #shift it win times
        df[str(HEADER,i)] = df[HEADER].shift(-i)
    
        
    return df
#unifinished

In [ ]:
def gru_model(gate,win,activation='relu'):
    model = keras.Sequential()
    model.add(keras.layers.Dense(win,activation,input_shape=(win,2,)))
    model.add(keras.layers.GRU(gate))
    model.add(keras.layers.Dense(2))
    model.summary()
    return model

In [ ]:
def lstm_model(gate,win,activation='relu'):
    model = keras.Sequential()
    model.add(keras.layers.Dense(win,activation,input_shape=(win,2,)))
    model.add(keras.layers.lstm(gate))
    model.add(keras.layers.Dense(2))
    model.summary()
    return model

In [ ]:
def simple_rnn_model(gate,win,activation='relu'):
    model = keras.Sequential()
    model.add(keras.layers.Dense(win,activation,input_shape=(win,2,)))
    model.add(keras.layers.SimpleRNN(gate))
    model.add(keras.layers.Dense(2))
    model.summary()
    return model

In [ ]:
def bidirectional_model(gate,win,activation='relu'): # experimental
    model = keras.Sequential()
    model.add(keras.layers.Dense(win,activation,input_shape=(win,2,)))
    model.add(keras.layers.Bidirectional(keras.layers.SimpleRNN(gate)))
    model.add(keras.layers.Dense(2))
    model.summary()
    return model

In [ ]:
def compile_train_test(model,X_train, X_val, Y_train, Y_val, X_test, Y_test, epochs, batches, loss='mse', opt='rmsprop', metrics):
    model.compile(optimizer=opt, loss = loss, metrics=metrics)
    model.summary()
    hist= model.fit(X_train,Y_train, epochs=epochs, batch_size= batches, validation_data=(X_val, Y_val))
    pred = model.evaluate(X_test,Y_test)
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    epochs = range(1, len(loss) +1 )
    plt.clf()
    plt.plot(epochs, loss, 'g.', label='Training loss')
    plt.plot(epochs, val_loss, 'b.', label='Validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

    #plt.clf()

    return model

